In [1]:
%matplotlib inline

In [2]:
import cv2 as cv
print(cv.__version__)

3.4.3


In [3]:
from imgaug import augmenters as iaa
import imgaug as ia

In [4]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [5]:
class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
            iaa.Scale((224, 224)),
            iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 1.3))),
            iaa.Fliplr(0.5),
            iaa.Affine(rotate=(-15, 15), mode='symmetric'),
            iaa.Sometimes(0.10,
                          iaa.OneOf([iaa.Dropout(p=(0, 0.03)),
                                     iaa.CoarseDropout(0.03, size_percent=0.2)])),
            iaa.AddToHueAndSaturation(value=(-20, 20), per_channel=True),
            iaa.PiecewiseAffine(),            
        ])
      
    def __call__(self, img):
        img = np.array(img)
        return self.aug.augment_image(img)

In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [7]:
transform_aug = transforms.Compose([
    ImgAugTransform(),
    lambda x: Image.fromarray(x),
    transforms.ToTensor(),
])

transform_resize_224 = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

In [8]:
def img_transform(src_root_path, dest_root_path):
    img1 = Image.open(src_root_path).convert('RGB')
    img2 = transform_aug(img1)
    transforms.ToPILImage()(img2).save(dest_root_path)
    
def img_transform_224(src_root_path, dest_root_path):
    img1 = Image.open(src_root_path).convert('RGB')
    img2 = transform_resize_224(img1)
    transforms.ToPILImage()(img2).save(dest_root_path)

In [16]:
count = 0

def traverse(src_root_path, dest_root_path, aug=False):
    global count
    
    dirs = os.listdir(src_root_path)
    for file in dirs:
        src_fullpath = os.path.join(src_root_path, file)
        dest_fullpath = os.path.join(dest_root_path, file)
        # print("src: {}, dest: {}".format(src_fullpath, dest_fullpath))

        if (os.path.isdir(src_fullpath) == True):  
            print("{} is a directory".format(src_fullpath)) 
            traverse(src_fullpath, dest_fullpath)
        
        elif os.path.isfile(src_fullpath):  
            # print("{} is a normal file".format(src_fullpath))
            if(aug == True):
                for i in range(10):
                    filename, file_extension = os.path.splitext(dest_fullpath)
                    img_transform(src_fullpath, "{}_{}{}".format(filename, i, file_extension))
            else:
                img_transform_224(src_fullpath, dest_fullpath)
               
            count += 1
            if((count % 20) == 19):
                print("progress: {}".format(count+1))
        
        else:  
            print("It is a special file (socket, FIFO, device file)")

#traverse("../data/cinic10/test", "../data/cinic10/test-224", False)
#traverse("../data/cinic10/valid", "../data/cinic10/valid-224", False)
#traverse("../data/cinic10/train", "../data/cinic10/aug", False)

#traverse("../data/cinic10/train/airplane", "../data/cinic10/aug/airplane", True)
#traverse("../data/cinic10/train/automobile", "../data/cinic10/aug/automobile", True)
#traverse("../data/cinic10/train/bird", "../data/cinic10/aug/bird", True)
#traverse("../data/cinic10/train/cat", "../data/cinic10/aug/cat", True)
#traverse("../data/cinic10/train/deer", "../data/cinic10/aug/deer", True)
#traverse("../data/cinic10/train/dog", "../data/cinic10/aug/dog", True)
#traverse("../data/cinic10/train/frog", "../data/cinic10/aug/frog", True)
#traverse("../data/cinic10/train/horse", "../data/cinic10/aug/horse", True)
#traverse("../data/cinic10/train/ship", "../data/cinic10/aug/ship", True)
#traverse("../data/cinic10/train/truck", "../data/cinic10/aug/truck", True)